In [38]:
from gwpy.timeseries import TimeSeries
from gwpy.timeseries import TimeSeriesDict

In [51]:
# The folder with O3GK C20 strain data and auxiliary channels
C20_archive = '/data/KAGRA/proc/C20/O3GK/K1'
full_archive = '/data/KAGRA/raw/full'
O3GK_start = 1270281600
O3GK_end = 1271462432

# Channel lists
strain_chanslist = [
    'K1:DAC-STRAIN_C20',
    # 'K1:DAC-DQ_STATE_VECTOR_C20'
]
rack_chanslist = [ 
    'K1:PEM-SENSOR_RACK_OMC1_DSUB1_OUT_DQ', 
    'K1:PEM-SENSOR_RACK_OMC1_DSUB2_OUT_DQ', 
    'K1:PEM-SENSOR_RACK_OMC1_DSUB3_OUT_DQ'
]
volt_chanslist = [
    'K1:PEM-VOLT_PSL_TABLE_GND_OUT_DQ', 
    'K1:PEM-VOLT_PSL_BOOTH_FLOAT_OUT_DQ', 
    'K1:PEM-VOLT_AS_TABLE_GND_OUT_DQ', 
    'K1:PEM-VOLT_OMC_CHAMBER_GND_OUT_DQ', 
    'K1:PEM-VOLT_REFL_TABLE_GND_OUT_DQ'
]
witness_chanslist = []
witness_chanslist.extend(rack_chanslist)
witness_chanslist.extend(volt_chanslist)

full_chanslist = []
full_chanslist.extend(strain_chanslist)
full_chanslist.extend(rack_chanslist)
full_chanslist.extend(volt_chanslist)

In [54]:
# Generating the list of gwf files to read
def gwf_source(source, start, end, f_duration):    
    start_dir = int(start/1e5)
    end_dir = int(end/1e5)
    gwf_list = []
    dir = start_dir
    while (dir <= end_dir):
        if (start >= int(dir*1e5)):
            f_start = int(start_dir*1e5 + f_duration*int((start - start_dir*1e5)/f_duration))
        else:
            f_start = int(dir*1e5)

        if (dir < end_dir):
            f_end = int(dir*1e5 + 99968)
        else:
            f_end = int(end_dir*1e5 + f_duration*int((end - end_dir*1e5)/f_duration))

        if (source.find('C20') != -1):
            for f_time in range(f_start, f_end + 1, f_duration):
                gwf_list.append(f'{source}/{dir}/K-K1_HOFT_C20-{f_time}-{f_duration}.gwf')
        elif (source.find('full') != -1):
            for f_time in range(f_start, f_end + 1, f_duration):
                gwf_list.append(f'{source}/{dir}/K-K1_C-{f_time}-{f_duration}.gwf')
        else:
            raise NameError('The source does not match "C20" or "full". Please check again the source.')
        dir += 1

    return gwf_list


In [58]:
start = 1270299780
end = 1270299780 + 36
ts_dict = TimeSeriesDict.fromkeys(full_chanslist)

strain = TimeSeries.read(
    source=gwf_source(C20_archive, start, end, 32),
    channel=strain_chanslist[0],
    start=start,
    end=end,
)
ts_dict[strain_chanslist[0]] = strain

for ch in witness_chanslist:
    ts = TimeSeries.read(
        source=gwf_source(full_archive, start, end, 32),
        channel=ch,
        start=start,
        end=end,
    )
    ts_dict[ch] = ts

In [72]:
st = int(ts_dict[full_chanslist[0]].t0.value)
du = int(ts_dict[full_chanslist[0]].duration.value)
ts_dict.write(f'/home/chia-jui.chou/DeepClean_O3GK/K-K1_DATA-{st}-{du}.gwf')